In [1]:
import os
import pandas as pd
import json
import numpy as np

# os.chdir('/Users/christineshearer/Desktop/workspace/GEM/Retirements Dashboard/data_wrangling_2')

In [ ]:
path_to_capacity_over_time = "./data_wrangling_2/raw_data/capacity_over_time.xlsx"
path_to_data_ticker = "./data_wrangling_2/raw_data/data_ticker.xlsx"
path_to_gap_in_ambition = "./data_wrangling_2/raw_data/gap_in_ambition.xlsx"
path_to_past_and_planned_retirement = "./data_wrangling_2/raw_data/past_and_planned_retirements.xlsx"
path_to_planned_retire_groupings = "./data_wrangling_2/raw_data/planned_retire_groupings.xlsx"

#set file names
project_data_dir = "./data"
year_filename = "h2_2024"
full_year = "2024"
year_variable = "H2 2024"

# Define where the public folder is.
path_to_public_assets_folder = "/Users/nicholasabad/Desktop/workspace/CodeOp/daft04/testing-repository/Phaseout-dashboard/public/assets"

if os.path.exists(path_to_public_assets_folder):
    
    # Define where the data folder is.
    path_to_data_folder = os.path.join(
        path_to_public_assets_folder, "data"
    )
    
    # If the folder does not exist yet, create it.
    if not os.path.exists(path_to_data_folder):
        os.mkdir(path_to_data_folder)
        print(f"Creating data folder at: {path_to_data_folder}")

In [3]:
# Function to generate the json files

def pandas_df_to_json(
    dataframe: pd.DataFrame,
    output_path: str,
):
    our_json_file = []
    for idx, row in dataframe.iterrows():
        curr_dict = {}
        for column in dataframe.columns:
            curr_dict[column] = str(row[column])
        
        our_json_file.append(curr_dict)
    
    with open(output_path, "w", encoding="utf-8") as output:
        json.dump(our_json_file, output, indent=4) 
        
    print(f"Wrote json file to {output_path}")

In [11]:
# 1. Capacity over time
capacity_over_time = pd.read_excel(path_to_capacity_over_time)
capacity_over_time = capacity_over_time.fillna("")

pandas_df_to_json(
    capacity_over_time, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_capacity_over_time.json")
)

Wrote json file to /Users/nicholasabad/Desktop/workspace/christine-projects/GEM/Retirements Dashboard/public/assets/data/gcpt_h2_2024_capacity_over_time.json


In [16]:
data_ticker

,Country/Area,Pledged coal phaseout year,1.5C pathway phaseout year,Total operating capacity (GW)
0,Global,2040,NaN,2126.000
1,G7,2030,2035.0,296.000
2,G20,2040,NaN,1968.000
3,EU27,2030,NaN,97.000
4,OECD,2030,NaN,443.000
...,...,...,...,...
109,Uzbekistan,2040,NaN,2.493
110,Venezuela,2040,NaN,0.000
111,Vietnam,2040,NaN,27.239
112,Zambia,2040,NaN,0.330


In [15]:
#To do: currently reading in "None" for coal phaseout as "nan"
# - the original data ticker code wrote out the text at the top of the dashboard and I think I will need to do the same with my data ticker code


# 2. Data ticker

data_ticker = pd.read_excel(path_to_data_ticker)
data_ticker["1.5C pathway phaseout year"] = data_ticker["1.5C pathway phaseout year"].fillna("None")

for idx, row in data_ticker.iterrows():
    pledged_coal_phaseout_year = row["Pledged coal phaseout year"]
    formatted_pledged_coal_phaseout_year = f"<span>{pledged_coal_phaseout_year}</span><br>Pledged Coal<br>Phaseout Year"
    data_ticker.loc[idx, "Pledged coal phaseout year"] = formatted_pledged_coal_phaseout_year
    
    net_zero_coal_phaseout_year = row["1.5C pathway phaseout year"]
    formatted_net_zero_coal_phaseout_year= f"<span>{net_zero_coal_phaseout_year}</span><br>1.5C Pathway<br>Phaseout Year"
    data_ticker.loc[idx, "1.5C pathway phaseout year"] = formatted_net_zero_coal_phaseout_year
    
    total_operating_capacity = row["Total operating capacity (GW)"]
    formatted_total_operating_capacity= f"<span>{{{{{total_operating_capacity}}}}}</span><br>Total Operating<br>Capacity (GW)"
    data_ticker.loc[idx, "Total operating capacity (GW)"] = formatted_total_operating_capacity


data_ticker.rename(
    {
        "Pledged coal phaseout year": "summary_1",
        "1.5C pathway phaseout year": "summary_2",
        "Total operating capacity (GW)": "summary_3"
    },
    axis=1,
    inplace=True
)

data_ticker["summary_1_color"] = "#bf532c"
data_ticker["summary_2_color"] = "#A8A8A8"
data_ticker["summary_3_color"] = "#f27d16"

data_ticker = data_ticker[["Country/Area", "summary_1", "summary_1_color", "summary_2", "summary_2_color", "summary_3", "summary_3_color"]]


pandas_df_to_json(
    data_ticker, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_data_ticker.json")
)


Wrote json file to /Users/nicholasabad/Desktop/workspace/CodeOp/daft04/testing-repository/Phaseout-dashboard/public/assets/data/gcpt_h2_2024_data_ticker.json


/var/folders/2s/38kg81v526j7qcgd8gdq8jd00000gn/T/ipykernel_5638/4284874246.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<span>2040</span><br>Pledged Coal<br>Phaseout Year' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_ticker.loc[idx, "Pledged coal phaseout year"] = formatted_pledged_coal_phaseout_year
/var/folders/2s/38kg81v526j7qcgd8gdq8jd00000gn/T/ipykernel_5638/4284874246.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<span>{{2126.0}}</span><br>Total Operating<br>Capacity (GW)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data_ticker.loc[idx, "Total operating capacity (GW)"] = formatted_total_operating_capacity


In [16]:
data_ticker

,Country/Area,summary_1,summary_1_color,summary_2,summary_2_color,summary_3,summary_3_color
0,Global,<span>2040</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>None</span><br>1.5C Pathway<br>Phaseout ...,#A8A8A8,<span>{{2126.0}}</span><br>Total Operating<br>...,#f27d16
1,G7,<span>2030</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>2035.0</span><br>1.5C Pathway<br>Phaseou...,#A8A8A8,<span>{{296.0}}</span><br>Total Operating<br>C...,#f27d16
2,G20,<span>2040</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>None</span><br>1.5C Pathway<br>Phaseout ...,#A8A8A8,<span>{{1968.0}}</span><br>Total Operating<br>...,#f27d16
3,EU27,<span>2030</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>None</span><br>1.5C Pathway<br>Phaseout ...,#A8A8A8,<span>{{97.0}}</span><br>Total Operating<br>Ca...,#f27d16
4,OECD,<span>2030</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>None</span><br>1.5C Pathway<br>Phaseout ...,#A8A8A8,<span>{{443.0}}</span><br>Total Operating<br>C...,#f27d16
...,...,...,...,...,...,...,...
109,Uzbekistan,<span>2040</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>None</span><br>1.5C Pathway<br>Phaseout ...,#A8A8A8,<span>{{2.493}}</span><br>Total Operating<br>C...,#f27d16
110,Venezuela,<span>2040</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>None</span><br>1.5C Pathway<br>Phaseout ...,#A8A8A8,<span>{{0.0}}</span><br>Total Operating<br>Cap...,#f27d16
111,Vietnam,<span>2040</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>None</span><br>1.5C Pathway<br>Phaseout ...,#A8A8A8,<span>{{27.239}}</span><br>Total Operating<br>...,#f27d16
112,Zambia,<span>2040</span><br>Pledged Coal<br>Phaseout ...,#bf532c,<span>None</span><br>1.5C Pathway<br>Phaseout ...,#A8A8A8,<span>{{0.33}}</span><br>Total Operating<br>Ca...,#f27d16


In [13]:
# 3. Gap in ambition
gap_in_ambition = pd.read_excel(path_to_gap_in_ambition)

for idx, row in gap_in_ambition.iterrows():
    value = row["Capacity"]
    if value >= 10:
        new_value = np.round(value, 0)
    elif 1 <= value < 10:
        new_value = np.round(value, 1)
    elif 0 < value < 1:
        new_value = np.round(value, 2)
    gap_in_ambition.loc[idx, "Capacity"] = new_value
    
    
pandas_df_to_json(
    gap_in_ambition, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_gap_in_ambition.json")
)

Wrote json file to /Users/nicholasabad/Desktop/workspace/christine-projects/GEM/Retirements Dashboard/public/assets/data/gcpt_h2_2024_gap_in_ambition.json


In [14]:
# 4. path_to_past_and_planned_retirement
past_and_planned_retirement = pd.read_excel(path_to_past_and_planned_retirement)
pandas_df_to_json(
    past_and_planned_retirement, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_past_and_planned_retirement.json")
)

Wrote json file to /Users/nicholasabad/Desktop/workspace/christine-projects/GEM/Retirements Dashboard/public/assets/data/gcpt_h2_2024_past_and_planned_retirement.json


In [15]:
# 5. path_to_planned_retire_groupings
planned_retire_groupings = pd.read_excel(path_to_planned_retire_groupings)

pandas_df_to_json(
    planned_retire_groupings, 
    os.path.join(path_to_data_folder, f"gcpt_{year_filename}_planned_retire_groupings.json")
)

Wrote json file to /Users/nicholasabad/Desktop/workspace/christine-projects/GEM/Retirements Dashboard/public/assets/data/gcpt_h2_2024_planned_retire_groupings.json
